# Part 2 : 数据清洗&指令数据集构建
<br>

### 任务目标
1. 利用gpt-4o、金山云瀚海，对爬取到原始资讯文章进行标注，按照不同标签进行相关性打分；
2. 数据清洗：由于爬取的文章token数过多、不均一，需要对其进行精简为500 token。使用TF-IDF对关键词提取，设计启发式规则进行过滤，最终精简为具有代表性的句子；
3. 按照Qwen微调指令集格式进行指令数据集组装构建。

<br>
<br>

In [65]:
import yaml
import pandas as pd
from IPython.display import display, clear_output
import mysql.connector
import json
import requests
import os
import random
from dotenv import load_dotenv

load_dotenv()

True

---
<br>

## 一、数据合成

指令格式化的数据实例包括任务描述（也称指令）、任务输入-任务输出以及可选的示例。指令数据集构建的方式一般包含基于现有的 NLP 任务数据集构建（如机器翻译、文本摘要和文本分类等）、基于日常对话数据构建

**本Demo中我们基于合成数据来构建，即通过性能更好的LLM（gpt-4o），输入更完整的全文信息，最终得出更精准的合成数据。**

### 1.1 连接MySQL获取原始爬取数据

In [66]:
with open(f'config_new.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [67]:
mysql_message = config['mysql']
connection = mysql.connector.connect(
        host=mysql_message['host'],
        port=mysql_message['port'],
        database=mysql_message['database'],
        user=mysql_message['user'],
        password=mysql_message['password'],
        ssl_disabled=True
    )

In [68]:
cursor = connection.cursor(dictionary=True)
query = """
            SELECT id, title, description_original
            FROM process_article_table
            LIMIT %s
            """
cursor.execute(query, (9999,))
query_list = cursor.fetchall()
display(pd.DataFrame(query_list)[:5])

,id,title,description_original
0,6,Create your fashion assistant application usin...,"In this post, we implement a fashion assistant..."
1,7,"How Aviva built a scalable, secure, and reliab...","In this post, we describe how Aviva built a fu..."
2,8,Visier’s data science team boosts their model ...,"In this post, we learn how Visier was able to ..."
3,11,Achieve operational excellence with well-archi...,"In this post, we discuss scaling up generative..."
4,12,Elevate workforce productivity through seamles...,"In this post, we explore how Amazon Q Business..."


<br>

### 1.2 利用OpenAI gpt-4o标注微调数据
由于国内无法直连OpenAI，需要代理。利用金山云新加坡云服务器部署Nginx，搭建OpenAI代理。北京——新加坡VPC通过对等连接实现内网打通，即可调用OpenAI。
```bash
sudo nano /etc/nginx/conf.d/openai_proxy.conf

    server {
            listen       3000;
            server_name  10.254.2.3;
        location / {
        proxy_pass https://api.openai.com/;
        proxy_ssl_server_name on;
        proxy_set_header Host api.openai.com;
        proxy_set_header Connection '';
        proxy_http_version 1.1;
        proxy_buffering off;
        proxy_cache off;
        proxy_set_header X-Forwarded-For $remote_addr;
        proxy_set_header X-Forwarded-Proto $scheme;
            }
        }
sudo nginx -t
sudo systemctl restart nginx
```

In [43]:
from openai import OpenAI


# 从环境变量中读取 API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY:
    print("OpenAI API Key loaded successfully!")
else:
    print("Failed to load OpenAI API Key.")

# 自定义 OpenAI API endpoint 配置
client = OpenAI(base_url="http://10.254.2.3:3000/v1")

OpenAI API Key loaded successfully!


In [44]:
def get_summary_and_tags(article):
    # 构造请求体
    system_content = f"""
    你将收到一篇包含文章信息的JSON，包含有id、title、description_original三个字段。

    你的任务是：
    1. 根据提供的标题和描述生成一个1-2句话的简短总结，放入description_ai_summary字段。
    2. 根据文章的title和description_original两个字段信息为如下每个标签的相关性打分（0~10分），放入category_relevance字段。
    标签列表：["计算机", "云计算", "人工智能", "架构师", "经济观察", "商业案例", "汽车行业", "个人娱乐", "其他"]。

    输出要求：
    1. 严格输出JSON格式，不要输出任何无关联信息，JSON的字段要求如下：
       - id: 保持和输入中的id一致
       - description_ai_summary: 1-2句话的总结，返回必须是中文
       - category_relevance: 对每个标签的相关性打分，分值在0到10之间
    """
    # 将article字典转换为JSON字符串
    article_str = json.dumps(article)
    # 调用OpenAI GPT-4 API
    response = client.chat.completions.create(
        model='gpt-4o-mini',  # 使用的模型
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": article_str}
        ],
        temperature=1,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    # 解析API返回结果
    result = response.choices[0].message
    return json.loads(result.content)

In [45]:
demo_result = get_summary_and_tags(query_list[0])
print(json.dumps(demo_result, indent=4, ensure_ascii=False))

{
    "id": 6,
    "description_ai_summary": "本文介绍了如何使用Amazon Bedrock代理和Amazon Titan模型构建一个时尚助手应用，提供个性化的多模态对话体验。",
    "category_relevance": {
        "计算机": 7,
        "云计算": 8,
        "人工智能": 9,
        "架构师": 4,
        "经济观察": 2,
        "商业案例": 5,
        "汽车行业": 1,
        "个人娱乐": 6,
        "其他": 3
    }
}


在这里为节省Token数，我们直接加载此前已经经过gpt-4o-mini标注过的数据集

In [69]:
cursor = connection.cursor(dictionary=True)
query = """
            SELECT title, description_original, description_ai_summary, category_relevance
            FROM process_article_table
            LIMIT %s
            """
cursor.execute(query, (9999,))
query_list = cursor.fetchall()
display(pd.DataFrame(query_list)[:5])

,title,description_original,description_ai_summary,category_relevance
0,Create your fashion assistant application usin...,"In this post, we implement a fashion assistant...",本文介绍了如何使用Amazon Bedrock代理和Titan模型创建一个时尚助手应用，该助...,"{""其他"": 3, ""云计算"": 7, ""架构师"": 6, ""计算机"": 8, ""个人娱乐""..."
1,"How Aviva built a scalable, secure, and reliab...","In this post, we describe how Aviva built a fu...",本文介绍了Aviva如何利用Amazon SageMaker构建一个可扩展、安全且可靠的ML...,"{""其他"": 3, ""云计算"": 9, ""架构师"": 7, ""计算机"": 8, ""个人娱乐""..."
2,Visier’s data science team boosts their model ...,"In this post, we learn how Visier was able to ...",这篇文章介绍了Visier如何通过迁移到Amazon SageMaker，将模型输出提升十倍...,"{""其他"": 1, ""云计算"": 9, ""架构师"": 7, ""计算机"": 8, ""个人娱乐""..."
3,Achieve operational excellence with well-archi...,"In this post, we discuss scaling up generative...",本文讨论了如何通过使用Amazon Bedrock，构建架构良好的生成式人工智能解决方案以实...,"{""其他"": 3, ""云计算"": 7, ""架构师"": 9, ""计算机"": 8, ""个人娱乐""..."
4,Elevate workforce productivity through seamles...,"In this post, we explore how Amazon Q Business...",本文探讨了Amazon Q Business如何通过个性化提升工作效率，并介绍了如何利用这一...,"{""其他"": 3, ""云计算"": 6, ""架构师"": 5, ""计算机"": 7, ""个人娱乐""..."


<br>

### 1.3 利用瀚海应用平台标注微调数据

![瀚海接入示意图](pictures/瀚海接入示意图.png)
注意：当前瀚海仅支持金山云VPN内网访问

In [7]:
HAN_HAI_APPKEY = os.getenv("HAN_HAI_APPKEY")
HAN_HAI_URL = "http://appbuilder.ai.ksyun.com/v1/chat-messages"

headers = {
    "Authorization": f"Bearer {HAN_HAI_APPKEY}",
    "Content-Type": "application/json"
}

data = {
    "query": query_list[0],
    "response_mode": "blocking",
    "conversation_id": "",
    "user": "rss-demo"
}

response = requests.post(HAN_HAI_URL, headers=headers, json=data)

# 打印响应结果
print(response.status_code)
print(response.json())

<module 'requests' from '/home/ubuntu/.local/lib/python3.10/site-packages/requests/__init__.py'>
404
{'error': {'type': 'invalid_request_error', 'code': 'unknown_url', 'message': 'Unknown request URL: POST /v1/chat-messages. Please check the URL for typos, or see the docs at https://platform.openai.com/docs/api-reference/.', 'param': None}}


---
<br>

## 二、数据清洗

为了有效控制输入的 token 数量，同时仍能为后续的标签打分任务提供充分的上下文信息，通过启发式策略进行关键词抽取与文本精简，以确保最终进入微调模型的文本长度可控，为后续的 LoRA 微调任务提供精简、聚焦但仍有代表性的输入数据。

	•	避免全文输入导致 token 爆炸与显存占用过大。
	•	能够在保留关键信息的前提下最大限度减少冗余，提升微调效率。
	•	借助启发式规则与传统NLP手段，无需依赖大型模型便可快速抽取关键词和核心信息。

#### 1.	关键词提取与核心句子抽取：
使用 TF-IDF、TextRank、YAKE 等传统文本挖掘算法从全文中自动提取最能代表主题的关键词和关键短语。根据提取出的关键词，进一步筛选出包含这些关键词的最关键句子，这些句子通常能代表文章的主体信息。将提炼出的关键词与核心句子拼接在一起构成精简版文本输入模型。

	•	使用 jieba、HanLP 或其他中文分词工具对文本进行分词。
	•	利用 TF-IDF 算法选出最高分的前20个关键词。
	•	使用 TextRank 提取前5~10句最能代表全文信息的句子。
	•	将关键词与这些句子合并（先列出关键词，然后列出选出的句子），这样生成一个简化后的文本输入模型。
#### 2.	基于规则的文本裁剪：
在得到关键词后，可以进一步对原文进行扫描，仅保留与关键词相关的上下文内容。比如：

	•	对每个关键词在原文中定位其出现位置，然后从该关键词周围截取一定长度（例如前后各100个字符）形成片段。
	•	将所有关键词相关片段合并去重后，再次做长度控制（如合并后超出最大字数，则仅保留出现频次最高的关键词片段）。

#### 3.	抽象摘要模型辅助：
如果条件允许，可在数据预处理阶段先用一个轻量级的摘要模型（如中文 T5 或 BERT 提示下的长文本摘要模型）对长文本生成短摘要。

	•	摘要模型会将冗长的文本浓缩为一段较短的描述。
	•	在摘要基础上再行关键词提取，以加强对文本主题的聚焦。
    
#### 4.	层级提炼策略：
对特别冗长的文本，可采用分段摘要-关键词提取的层级策略：

	•	将全文分割成若干小段（每段适合摘要模型处理或 tokenizer 的限制范围内）。
	•	对每段独立生成关键词和简短摘要，再将所有摘要合并为一个更短的「汇总摘要」并附上该汇总摘要中的关键词。
	•	这样可将任意长度的文本分段迭代压缩、提炼，直到得到可以直接输入到微调模型内的短文本。
    
#### 5.	启发式过滤与删减：
可以根据任务标签类别和背景知识设定启发式过滤规则。例如，如果标签涉及「计算机、云计算、人工智能、架构师、经济观察、商业案例、汽车行业、个人娱乐、其他」，在文本预处理时，就优先选取与这些领域相关的术语或段落，对于明显与目标分类不相关的描述（如冗长的历史背景、与主题不相关的闲笔）进行删除。

In [47]:
import jieba
import jieba.analyse

<br>

### 2.1 分词

In [48]:
# 示例长文本（请用您自己的数据替换此长文本）
long_text = """
如今，人工智能在企业技术优先事项中的地位迅速上升。数据报告显示，2024 年第二季度，首席执行官在财报电话会议上讨论人工智能的次数超过了任何其他技术主题，达到 34%——当 CEO 谈论 AI 时，他们的团队会开始部署云端 AI 技术。

20219 年第一季度至 2024 年第三季度，人工智能的衰落与崛起是 CEO 们常见的讨论话题（图源：IoT Analytics）
根据 AWS 的定义，AI 云是在传统云计算基础上增加了人工智能相关功能和服务，使得开发者和企业能够更轻松地构建、部署和管理人工智能应用程序。使用 AI 云服务，企业和开发者无需自行搭建庞大的基础设施和人工智能系统，就可以快速构建和部署 AI 应用。AI 云可以提供机器学习、语音识别和合成、自然语言处理、计算机视觉、智能推荐系统、预测分析、虚拟助手、自动化决策、数据分析和挖掘等服务……
2024 年 10 月，物联网研究机构 IoT Analytics 发布了一份长达 188 页的《2024 年全球云项目报告和数据库》报告，其中提到一些核心洞察：
最近宣布的云实施项目中有22% 包含 AI 元素。数据显示，AI 已成为云需求的驱动力，而生成式 AI (GenAI)发挥着越来越重要的作用。
从企业的维度来看，微软在整体 AI 和 GenAI 竞赛中处于领先地位，AWS在传统 AI 领域处于领先地位，而谷歌在 AI 客户中所占份额最高。
数据透视 2024 云端 AI 项目现状
《2024 年全球云项目报告和数据库》分析了近年来最大的云供应商的 8300 多个客户实施项目（包括对 AI 和 IoT 云项目的深入研究），结果显示，2023 年 6 月至 2024 年 6 月期间，全球前五大超大规模企业发布了 2700 多个新客户案例研究，其中 608 个使用云 AI 服务（22%）。
根据 IoT Analytics 自己的模型，下表统计了 2023 年公共云市场份额：

表中的 “其他”包括Oracle和阿里巴巴；但鉴于它们在云端 AI 项目中的份额较小，本文的重点将放前三大超大规模企业，即 AWS、微软和谷歌身上，并比较他们三者的最新云 AI 产品和项目：

①整体云端 AI 项目
AI 案例研究总数：微软新增了最多的 AI 案例研究，在 608 个新的云端 AI 案例研究中，微软的案例研究数量最多，达到了 274 个（占 608 的 45%），分析期间的新 AI 客户包括保险巨头 AXA、专业服务公司 KPMG、工业自动化公司施耐德电气和啤酒公司喜力；AWS在这方面排名第二，拥有 207 个案例（占 34%），电子巨头三星和食品饮料制造商雀巢是 AWS 新增的两家主要 AI 客户；谷歌以 102 个案例（占 17%）排名第三，新增客户包括制药公司默克和旅行社 Priceline.com。AWS 在传统 AI 案例研究中数量最高，即不包括生成式 AI，共有 176 个。值得注意的是，谷歌的新 AI 案例研究占所有新云案例研究的比例最高，102 个占 280 个（占36%）。
AI 案例研究相对市场份额的数量：微软在新案例研究的份额与其市场份额之间的差距最大；AWS的新云端 AI 案例研究的份额（34%）比其 2023 年云市场份额的 37% 低 3 个百分点；谷歌的案例研究份额为 17%，比其 9% 的云市场份额高出 8 个百分点，而微软的案例研究份额为 45%，比其 29% 的云市场份额高出 16 个百分点。因此，微软和谷歌的表现均优于其云市场份额。
②云端生成式 AI 项目
全球云项目报告和数据库包括 206 个新的生成式 AI 案例研究，占所有 AI 案例研究的 34%，其中：
微软在云端生成式 AI 案例研究中领先，206 个云端生成式 AI 案例研究中，微软明显领先，拥有127个案例（占 206 的 62%）；谷歌排名第二，拥有 37 个案例（占 18%）；AWS则有 33 个（占 16%）。
AWS的新云端生成式 AI 案例研究份额（16%）比其 2023 年云市场份额的 37% 低 21 个百分点；谷歌的案例研究份额为 18%，比其 9% 的云市场份额高出 9个百分点；而微软62% 的新生成式 AI 案例研究份额比其 29% 的云市场份额高出 33 个百分点。这再次表明，微软在云 AI 竞争中表现优于同行。
谁将赢得云端 AI 竞赛？
基于调研数据和分析师洞察，我们也能初步得出一些结论：
① 目前，微软在云端 AI 竞赛和 GenAI 竞赛中处于领先地位
就目前而言，当将新的云端 AI 案例研究与整体云端研究进行比较时，微软在竞赛中处于明显领先地位，并且可能在短期内保持领先地位。
得益于与 OpenAI 的密切关系，微软在云端 GenAI 中取得了显著的领先优势。早从 2019 年，微软就是 OpenAI 的大力支持者，并在 2023 年 1 月，也就是 ChatGPT 公开发布不到两个月事后，微软宣布了对 OpenAI “多年、数十亿美元”的投资计划；2023 年 4 月，微软再次加码，对 OpenAI 额外投资 100 亿美元，持股比例达到 49%，并规定了微软能够获得 OpenAI 利润的一定比例，直到收回投资为止；在 OpenAI 最新一轮高达 66 亿美元的融资中，微软再度出资 7.5 亿美元。总体来看，美国三大云巨头中，微软是对 AI 投入最为果决、也最全面的一家。
许多大型企业已经在 Microsoft AI 堆栈之上启动了他们的第一个 GenAI 项目。但是，随着几个 LLM 正在缩小与 OpenAI 模型的性能差距，微软的先发优势是会消失还是会继续存在还有待观察。
而且，也有媒体报道称 OpenAI 与微软的合作正因财务分歧、资源争夺和战略冲突而变得紧张。据《纽约时报》的一篇长篇报道，OpenAI 的 CEO Sam Altman 向微软的 Satya Nadella 寻求额外支持时，微软因对成本上升的担忧而犹豫不决。OpenAI 预计今年将亏损 50 亿美元，并预计到 2028 年将亏损 440 亿美元，这主要是因为开发先进 AI 系统涉及的高昂费用。面对财务压力，为了减少对微软的依赖，OpenAI 一直也在寻求新的投资者和合作伙伴。
② AWS 在传统 AI 领域处于领先地位
如果将视线转到没有 GenAI 元素的云 AI 案例研究方面，我们会发现 AWS 在传统云 AI 方面处于领先地位。
与此同时，AWS 的 Amazon SageMaker 是云 AI 案例研究中涵盖最多的产品，占 21%。据介绍，Amazon SageMaker 是一种全面托管的机器学习服务。这意味着用户无需过多关注底层的复杂性，而是可以专注于模型的构建和优化。通过 SageMaker，开发者可以快速、轻松地构建和训练机器学习模型，然后直接将模型部署到生产就绪的托管环境中。同时 SageMaker 也提供了一个集成的 Jupyter 编写 Notebook 实例，可以轻松访问数据源以便进行探索和分析，并且无需管理服务器。

随着 AWS 进一步开发其云 AI 产品（尤其是 GenAI），它可以开始向其现有和新客户群推销这些服务，看到更多的云 AI 项目，并获得更大的新项目份额。
③谷歌拥有最高的 AI 客户份额
谷歌历来受到小型公司的青睐，在云 AI 竞赛中也有自己的优势。在查看每个供应商的新案例研究时，谷歌的云 AI 案例研究占其整体新案例研究数量的比重最大。谷歌 36% 的新公共云案例研究使用了云 AI 产品，这意味着 AI 对谷歌云的推动作用比对其他任何超大规模企业都大。

云安全服务至关重要
报告还提及，在不考虑新的云 AI 案例研究的情况下，云安全服务是过去一年增长最快的产品之一。典型产品比如Microsoft Sentinel，其是一个云原生的 SIEM 和 SOAR 服务。SIEM 指的是安全信息和事件管理系统，便于安全团队从硬件设备或应用程序上收集日志，分析日志的可疑情况，再由可疑情况，生成警报和创建case；SOAR 以安全编排和自动化为核心，辅助安全运营人员日常工作，提升安全运营效率。
笔者在此前的多篇文章中也强调——近年来，随着各大公司投入巨资将数据转移到云平台，云安全变得尤为重要。脸书、谷歌、微软和亚马逊等公司收集了大量数据，从行为模式和用户偏好到专有算法和敏感的企业数据，这些已成为网络犯罪分子非常有利可图的目标。仅去年一年，诺顿、MailChimp、X、威瑞森、谷歌、动视、ChatGPT、T-Mobile、微软、沃尔玛、三星、富士通、美国运通等就都遭遇了网络攻击。
而在今年 7 月的时候，谷歌母公司 Alphabet 正就约 230 亿美元收购云计算网络安全初创公司 Wiz 进行深入谈判，若交易达成，可能将造就谷歌史上规模最大的一次收购。作为全球领先的科技巨头，谷歌的重要战略性举措往往会成为整个科技圈的风向标，其对 Wiz 所下的重注，正反映了当下火热发展的云安全市场。
不少研究机构预测，开发创新方案来应对安全挑战的云安全公司将成为 2024 年网络安全行业中增长最快且价值最高的公司之一。
参考资料
《Who is winning the cloud AI race? Microsoft vs. AWS vs. Google》，iot-analytics
《Amazon SageMaker：让机器学习变得更简单、更强大》，亚马逊云开发者
《OpenAI与微软合作面临破裂风险：神秘AGI条款或成导火索》，水哥
《230亿！谷歌开出有史以来最高收购天价！云安全市场爆了》，物联网智库
本文来自微信公众号“物联网智库”（ID：iot101），作者：Sophia，36氪经授权发布。"""

In [71]:
# jieba分词可根据需求启用自定义词典（该字典仅能保证正确被分词）
jieba.load_userdict("datacleaning/High-frequency-vocabulary-jieba.txt")
# 对文本进行基础清洗（如去除多余空格、换行符等）
cleaned_text = long_text.strip().replace('\n', '').replace('\r', '')

print(jieba.lcut(cleaned_text)[:50]) #仅展示前50

['如今', '，', '人工智能', '在', '企业', '技术', '优先', '事项', '中', '的', '地位', '迅速', '上升', '。', '数据', '报告', '显示', '，', '2024', ' ', '年', '第二季度', '，', '首席', '执行官', '在', '财报', '电话会议', '上', '讨论', '人工智能', '的', '次数', '超过', '了', '任何', '其他', '技术', '主题', '，', '达到', ' ', '34%', '—', '—', '当', ' ', 'CEO', ' ', '谈论']


<br>

### 2.2 关键词提取（TF-IDF）
TF-IDF = TF * IDF

	•	TF（Term Frequency）：词在文本中出现的次数。出现一次的词其TF相对较低。
	•	IDF（Inverse Document Frequency）：值越高表示此词越稀有、越有区分度；值越低表示此词过于常见。

参数topK可根据文本长度和需求适当调整，如关键词过少信息不够，过多会冗余

In [50]:
jieba.analyse.set_idf_path("datacleaning/High-frequency-vocabulary-IDF.txt")
top_keywords = jieba.analyse.extract_tags(cleaned_text, topK=20, withWeight=False)
print(top_keywords)

['AI', '案例', '研究', '微软', '份额', '谷歌', '云端', 'AWS', '项目', 'Open', '安全', '企业', '服务', '公司', '2024', '生成式', '云市场', '数据', 'Gen', '领先地位']


In [51]:
keyword_weights = {k: i for i, k in enumerate(top_keywords[::-1], start=1)}
# 上面将关键词反向排序，以最高权重为最大值

print(keyword_weights)

{'领先地位': 1, 'Gen': 2, '数据': 3, '云市场': 4, '生成式': 5, '2024': 6, '公司': 7, '服务': 8, '企业': 9, '安全': 10, 'Open': 11, '项目': 12, 'AWS': 13, '云端': 14, '谷歌': 15, '份额': 16, '微软': 17, '研究': 18, '案例': 19, 'AI': 20}


<br>

### 2.3 分句
topK可根据长度调优，一般选取5~10句即可，jieba的textrank默认用于关键词提取，如需句子提取，可自行对原文分句后调用textrank关键词后再映射回句子，或使用其他句子抽取工具。

In [72]:
# key_sentences = jieba.analyse.textrank(cleaned_text, topK=10, withWeight=False, allowPOS=('ns','n','vn','v','nr','nt','nz','a','d'))

# 句子分句函数（可根据标点适当扩展）
def split_sentences(text):
    import re
    # 中文分句标点符号列表可根据实际情况微调
    sentences = re.split(r'[。！？!?]', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences

sent_list = split_sentences(cleaned_text)
# 创建 DataFrame
pd.DataFrame(sent_list, columns=["分句结果"])[:5].style.set_properties(**{'text-align': 'left'}) #仅展示前5句

,分句结果
0,如今，人工智能在企业技术优先事项中的地位迅速上升
1,数据报告显示，2024 年第二季度，首席执行官在财报电话会议上讨论人工智能的次数超过了任何其他技术主题，达到 34%——当 CEO 谈论 AI 时，他们的团队会开始部署云端 AI 技术
2,20219 年第一季度至 2024 年第三季度，人工智能的衰落与崛起是 CEO 们常见的讨论话题（图源：IoT Analytics）根据 AWS 的定义，AI 云是在传统云计算基础上增加了人工智能相关功能和服务，使得开发者和企业能够更轻松地构建、部署和管理人工智能应用程序
3,使用 AI 云服务，企业和开发者无需自行搭建庞大的基础设施和人工智能系统，就可以快速构建和部署 AI 应用
4,AI 云可以提供机器学习、语音识别和合成、自然语言处理、计算机视觉、智能推荐系统、预测分析、虚拟助手、自动化决策、数据分析和挖掘等服务……2024 年 10 月，物联网研究机构 IoT Analytics 发布了一份长达 188 页的《2024 年全球云项目报告和数据库》报告，其中提到一些核心洞察：最近宣布的云实施项目中有22% 包含 AI 元素


<br>

### 2.4 启发式过滤，找到最匹配句子
保留与任务领域相关的句子
策略：句子分词后，如包含任一domain_keywords_set中关键词则保留

In [53]:
# 定义领域相关关键词词典(根据标签类别定制)，重点聚焦这些类别相关的文本:
domain_keywords = [
    "计算机","云计算","人工智能","架构师","经济观察","商业案例",
    "汽车行业","个人娱乐","芯片","CPU","GPU",
    "经济","商业","英伟达","计算","存储","网络","AI","算力"
]

In [54]:
# 根据关键词权重对句子进行打分排序
def sentence_rank(sentences_list, keyword_weights):
    """
    :param sentences_list: 待排序句子的列表
    :param keyword_weights: 关键词权重计算函数
    :return: 按得分排序的句子列表
    """
    def sentence_score(sent):
            words = jieba.lcut(sent)
            score = sum(keyword_weights.get(w, 0) for w in words)
            return score

    # 对句子进行评分并排序
    scored_sentences = [(s, sentence_score(s)) for s in sentences_list]
    scored_sentences.sort(key=lambda x: x[1], reverse=True)
    return [s for s, _ in scored_sentences]

In [55]:
# 根据领域关键词筛选句子并按优先级排序，保证至少返回N个句子
def filter_and_rank_sentences(sent_list, keyword_weights, domain_keywords, N=6):
    """
    :param sent_list: 所有句子的列表
    :param keyword_weights: 关键词权重计算函数
    :param domain_keywords: 领域相关关键词列表
    :param N: 返回的句子数量
    :return: 筛选并排序后的句子列表
    """
    # 转换领域关键词为集合
    domain_keywords_set = set(domain_keywords)

    # 过滤包含领域关键词的句子
    filtered_sentences = [
        s for s in sent_list
        if any(w in domain_keywords_set for w in jieba.lcut(s))
    ]

    # 对过滤的句子进行排序
    ranked_filtered_sentences = sentence_rank(filtered_sentences, keyword_weights)

    # 如果不足N个句子，从所有句子中补充
    if len(ranked_filtered_sentences) < N:
        # 对所有句子排序
        ranked_all_sentences = sentence_rank(sent_list, keyword_weights)
        # 补充缺失的句子
        additional_needed = N - len(ranked_filtered_sentences)
        additional_sentences = [
            s for s in ranked_all_sentences if s not in ranked_filtered_sentences
        ][:additional_needed]
        ranked_filtered_sentences.extend(additional_sentences)

    # 返回最终的N个句子
    return ranked_filtered_sentences[:N]

In [56]:
# 运行筛选和排序
result_sentences = filter_and_rank_sentences(sent_list, keyword_weights, domain_keywords, N=6)

# 创建 DataFrame
pd.DataFrame(result_sentences, columns=["筛选结果"]).style.set_properties(**{'text-align': 'left'})

,筛选结果
0,根据 IoT Analytics 自己的模型，下表统计了 2023 年公共云市场份额：表中的 “其他”包括Oracle和阿里巴巴；但鉴于它们在云端 AI 项目中的份额较小，本文的重点将放前三大超大规模企业，即 AWS、微软和谷歌身上，并比较他们三者的最新云 AI 产品和项目：①整体云端 AI 项目AI 案例研究总数：微软新增了最多的 AI 案例研究，在 608 个新的云端 AI 案例研究中，微软的案例研究数量最多，达到了 274 个（占 608 的 45%），分析期间的新 AI 客户包括保险巨头 AXA、专业服务公司 KPMG、工业自动化公司施耐德电气和啤酒公司喜力；AWS在这方面排名第二，拥有 207 个案例（占 34%），电子巨头三星和食品饮料制造商雀巢是 AWS 新增的两家主要 AI 客户；谷歌以 102 个案例（占 17%）排名第三，新增客户包括制药公司默克和旅行社 Priceline.com
1,②云端生成式 AI 项目全球云项目报告和数据库包括 206 个新的生成式 AI 案例研究，占所有 AI 案例研究的 34%，其中：微软在云端生成式 AI 案例研究中领先，206 个云端生成式 AI 案例研究中，微软明显领先，拥有127个案例（占 206 的 62%）；谷歌排名第二，拥有 37 个案例（占 18%）；AWS则有 33 个（占 16%）
2,AI 案例研究相对市场份额的数量：微软在新案例研究的份额与其市场份额之间的差距最大；AWS的新云端 AI 案例研究的份额（34%）比其 2023 年云市场份额的 37% 低 3 个百分点；谷歌的案例研究份额为 17%，比其 9% 的云市场份额高出 8 个百分点，而微软的案例研究份额为 45%，比其 29% 的云市场份额高出 16 个百分点
3,AWS的新云端生成式 AI 案例研究份额（16%）比其 2023 年云市场份额的 37% 低 21 个百分点；谷歌的案例研究份额为 18%，比其 9% 的云市场份额高出 9个百分点；而微软62% 的新生成式 AI 案例研究份额比其 29% 的云市场份额高出 33 个百分点
4,早从 2019 年，微软就是 OpenAI 的大力支持者，并在 2023 年 1 月，也就是 ChatGPT 公开发布不到两个月事后，微软宣布了对 OpenAI “多年、数十亿美元”的投资计划；2023 年 4 月，微软再次加码，对 OpenAI 额外投资 100 亿美元，持股比例达到 49%，并规定了微软能够获得 OpenAI 利润的一定比例，直到收回投资为止；在 OpenAI 最新一轮高达 66 亿美元的融资中，微软再度出资 7.5 亿美元
5,基于调研数据和分析师洞察，我们也能初步得出一些结论：① 目前，微软在云端 AI 竞赛和 GenAI 竞赛中处于领先地位就目前而言，当将新的云端 AI 案例研究与整体云端研究进行比较时，微软在竞赛中处于明显领先地位，并且可能在短期内保持领先地位


<br>

### 2.5 合并关键词与精简句子

In [57]:
final_text = "关键词: " + " ".join(top_keywords) + "\n" + "核心句子:\n" + "\n".join(result_sentences)

print(final_text)

关键词: AI 案例 研究 微软 份额 谷歌 云端 AWS 项目 Open 安全 企业 服务 公司 2024 生成式 云市场 数据 Gen 领先地位
核心句子:
根据 IoT Analytics 自己的模型，下表统计了 2023 年公共云市场份额：表中的 “其他”包括Oracle和阿里巴巴；但鉴于它们在云端 AI 项目中的份额较小，本文的重点将放前三大超大规模企业，即 AWS、微软和谷歌身上，并比较他们三者的最新云 AI 产品和项目：①整体云端 AI 项目AI 案例研究总数：微软新增了最多的 AI 案例研究，在 608 个新的云端 AI 案例研究中，微软的案例研究数量最多，达到了 274 个（占 608 的 45%），分析期间的新 AI 客户包括保险巨头 AXA、专业服务公司 KPMG、工业自动化公司施耐德电气和啤酒公司喜力；AWS在这方面排名第二，拥有 207 个案例（占 34%），电子巨头三星和食品饮料制造商雀巢是 AWS 新增的两家主要 AI 客户；谷歌以 102 个案例（占 17%）排名第三，新增客户包括制药公司默克和旅行社 Priceline.com
②云端生成式 AI 项目全球云项目报告和数据库包括 206 个新的生成式 AI 案例研究，占所有 AI 案例研究的 34%，其中：微软在云端生成式 AI 案例研究中领先，206 个云端生成式 AI 案例研究中，微软明显领先，拥有127个案例（占 206 的 62%）；谷歌排名第二，拥有 37 个案例（占 18%）；AWS则有 33 个（占 16%）
AI 案例研究相对市场份额的数量：微软在新案例研究的份额与其市场份额之间的差距最大；AWS的新云端 AI 案例研究的份额（34%）比其 2023 年云市场份额的 37% 低 3 个百分点；谷歌的案例研究份额为 17%，比其 9% 的云市场份额高出 8 个百分点，而微软的案例研究份额为 45%，比其 29% 的云市场份额高出 16 个百分点
AWS的新云端生成式 AI 案例研究份额（16%）比其 2023 年云市场份额的 37% 低 21 个百分点；谷歌的案例研究份额为 18%，比其 9% 的云市场份额高出 9个百分点；而微软62% 的新生成式 AI 案例研究份额比其 29% 的云市场份额高出 33 个百分点
早从 2019 年，微软就是 OpenAI 的大力支持者

<br>

### 2.6 文本长度控制
如果此时final_text仍过长，您可以对字符长度进行截断

In [58]:
MAX_CHARS = 500  # 假设我们希望不超过500个字符
if len(final_text) > MAX_CHARS:
    final_text = final_text[:MAX_CHARS] + "..."

print("最终处理后的文本片段：\n", final_text)

最终处理后的文本片段：
 关键词: AI 案例 研究 微软 份额 谷歌 云端 AWS 项目 Open 安全 企业 服务 公司 2024 生成式 云市场 数据 Gen 领先地位
核心句子:
根据 IoT Analytics 自己的模型，下表统计了 2023 年公共云市场份额：表中的 “其他”包括Oracle和阿里巴巴；但鉴于它们在云端 AI 项目中的份额较小，本文的重点将放前三大超大规模企业，即 AWS、微软和谷歌身上，并比较他们三者的最新云 AI 产品和项目：①整体云端 AI 项目AI 案例研究总数：微软新增了最多的 AI 案例研究，在 608 个新的云端 AI 案例研究中，微软的案例研究数量最多，达到了 274 个（占 608 的 45%），分析期间的新 AI 客户包括保险巨头 AXA、专业服务公司 KPMG、工业自动化公司施耐德电气和啤酒公司喜力；AWS在这方面排名第二，拥有 207 个案例（占 34%），电子巨头三星和食品饮料制造商雀巢是 AWS 新增的两家主要 AI 客户；谷歌以 102 个案例（占 17%）排名第三，新增客户包括制药公司默克和旅行社 Priceline.com
②云端生成式 AI 项目...


---
<br>

## 三、微调数据集组装

### 3.1 指令数据集构建介绍
#### 1. Qwen2.5-7B指令集模版

LLM 的微调一般指指令微调过程，我们的核心训练目标是让模型具有理解并遵循用户指令的能力。因此，在指令集构建时，我们应针对我们的目标任务，针对性构建任务指令集。本项目以文章标签打分为例，我们的目标是构建一个当输入的文章简介，能根据内容进行个性化打标签的LLM，因此我们构造的指令形如

```json
{
  "instruction": "将以下文本按照【其他、云计算、架构师、计算机、个人娱乐、人工智能、商业案例、汽车行业、经济观察】标签纬度进行关联度评分",
  "input": "直面创业真命题：消费降级了吗？不要错过十亿人的大市场
    关键词: 我们 品牌 消费 一个 行业 如果 思维 国民 10 创始人 很多 红利 品质 降级 机会 时候 市场 创新 创业 因为
    核心句子:回想一下，日本经济泡沫破灭之后，出现了大量类似于萨莉亚、优衣库等国民品牌，沃尔玛开始起飞的时机也是在美国经济大危机之后
    AI会彻底改变白领行业，促使人才结构由金字塔型向松树塔型演进
    有了AI之后，涌现出了很多小型的超级团队，这些团队基本上由5-10人组成，并且往往做的是B端
    对于创业型公司来说，AI使得人员数量比常规状态下减少了一半以上
    与餐饮行业的情况相同，短视频+AI+机器人的范式将会彻底改变整个消费领域
    如果用AI+短视频的模式构建一个产品的核心能力，你就会在某个场景或某个细分市场里成为领导品牌，甚至成为国民品牌",
  "output": '
    {
      "其他": 6, 
      "云计算": 0, 
      "架构师": 0, 
      "计算机": 0, 
      "个人娱乐": 0, 
      "人工智能": 9, 
      "商业案例": 8, 
      "汽车行业": 0, 
      "经济观察": 9, 
      "经济观察": 9
    }'
}
```
其中，`instruction` 是用户指令，告知模型其需要完成的任务；`input` 是用户输入，是完成用户指令所必须的输入内容；`output` 是模型应该给出的输出。

#### 2. 指令数据提升方法
##### 指令格式设计，优化instruction任务描述
在指令微调中，指令数据的格式、数量等因素对微调后的模型性能有着重要影响。其中指令格式的设计对模型性能有重大影响，通常来说，可以直接向现有数据集的输入-输出对上添加任务描述构建指令微调数据，这其中任务描述是大模型理解任务的关键部分。

在本Demo中，我们发现，当任务instruction写为`将以下文本分类`时，模型效果很差，不能精准的完成任务目标（只从给定的标签来打分，不生成新的标签）。

![instruction错误示范](pictures/instruction错误示范.png)

当优化为instruction = `将以下文本按照【其他、云计算、架构师、计算机、个人娱乐、人工智能、商业案例、汽车行业、经济观察】标签纬度进行关联度评分`后，模型任务完成率接近100%，有大幅提升


##### 优化数据组织策略
指令微调过程中还需要考虑一定的数据组织形式，从而使得模型获得更好的微调效果。一般可通过两种策略

1. 平衡数据分布，混合使用不同来源数据，如NLP，对话，合成数据，可以综合提升模型全方位能力。

2. 结合预训练数据与指令微调数据，例如在指令微调阶段引入少量（5%）的预训练数据，在分类和生成任务上都能取得增益；增加预训练数据会对生成任务有利，但有可能损失分类任务的表现。通过提前使用指令微调数据，有可能会帮助模型在预训练阶段更好地感知下游任务，从而更针对性地从预训练数据中学习知识与能力。例如，GLM-130B的预训练过程由95%的传统自监督预训练和 5% 的指令微调任务混合组成。

3. 多阶段指令数据微调首先使用大规模 NLP 任务指令数据对模型进行微调，然后再使用相对多样的日常对话指令和合成指令进一步微调。为了避免能力遗忘问题，可以在第二阶段中添加一些 NLP 指令数据。这种多阶段的微调策略也可以应用于其他训练设置。例如，对于不同的微调阶段，训练中可以逐渐增加指令的难度和复杂性，从而逐渐提高大模型遵循复杂指令的能力。

<br>

### 3.2 数据集组装

In [59]:
# 我们将2逻辑合并，封装为自定义清洗库后引入
from datacleaning.data_cleaning_filter import sentences_filter

In [60]:
my_dataset = []

# 遍历处理数据
for i, row in enumerate(query_list[:50], start=1):  # [:50] Demo演示中，我们只组装前50条数据
    # 调用过滤逻辑
    description = row['description_original']
    if len(description) > 500:
        description = sentences_filter(description)

    # 组装微调数据
    instruction = "将以下文本按照【其他、云计算、架构师、计算机、个人娱乐、人工智能、商业案例、汽车行业、经济观察】标签纬度进行关联度评分"
    input_text = f"{row['title']} {description}"
    output_text = row['category_relevance']

    # 将结果加入数据集
    my_dataset.append({
        "instruction": instruction,
        "input": input_text,
        "output": output_text
    })

    # 实时输出进度
    clear_output(wait=True)  # 清除上一条输出
    print(f"处理进度：{i}/{len(query_list)}")
    print("当前处理数据：")
    print(json.dumps({
        "instruction": instruction,
        "input": input_text,
        "output": output_text
    }, ensure_ascii=False, indent=4))  # 格式化 JSON 输出，保持中文
    
print("数据集组装成功！")

处理进度：50/4331
当前处理数据：
{
    "instruction": "将以下文本按照【其他、云计算、架构师、计算机、个人娱乐、人工智能、商业案例、汽车行业、经济观察】标签纬度进行关联度评分",
    "input": "Gemma: Introducing new state-of-the-art open models Gemma is built for responsible AI development from the same research and technology used to create Gemini models.",
    "output": "{\"其他\": 3, \"云计算\": 5, \"架构师\": 6, \"计算机\": 8, \"个人娱乐\": 1, \"人工智能\": 10, \"商业案例\": 4, \"汽车行业\": 1, \"经济观察\": 2}"
}
数据集组装成功！


In [73]:
display(pd.DataFrame(my_dataset)[:5])

,instruction,input,output
0,将以下文本按照【其他、云计算、架构师、计算机、个人娱乐、人工智能、商业案例、汽车行业、经济观...,How Northpower used computer vision with AWS t...,"{""其他"": 1, ""云计算"": 8, ""架构师"": 3, ""计算机"": 9, ""个人娱乐""..."
1,将以下文本按照【其他、云计算、架构师、计算机、个人娱乐、人工智能、商业案例、汽车行业、经济观...,Looking ahead to the AI Seoul Summit How summi...,"{""其他"": 2, ""云计算"": 3, ""架构师"": 5, ""计算机"": 8, ""个人娱乐""..."
2,将以下文本按照【其他、云计算、架构师、计算机、个人娱乐、人工智能、商业案例、汽车行业、经济观...,"Gemini breaks new ground: a faster model, long...","{""其他"": 3, ""云计算"": 6, ""架构师"": 5, ""计算机"": 9, ""个人娱乐""..."
3,将以下文本按照【其他、云计算、架构师、计算机、个人娱乐、人工智能、商业案例、汽车行业、经济观...,Empowering YouTube creators with generative AI...,"{""其他"": 2, ""云计算"": 4, ""架构师"": 3, ""计算机"": 7, ""个人娱乐""..."
4,将以下文本按照【其他、云计算、架构师、计算机、个人娱乐、人工智能、商业案例、汽车行业、经济观...,"New generative media models and tools, built w...","{""其他"": 2, ""云计算"": 5, ""架构师"": 4, ""计算机"": 8, ""个人娱乐""..."


<br>

### 3.3 训练、评测数据集拆分

In [62]:
# 随机打乱数据
random.shuffle(my_dataset)

# 计算训练集和评测集的分割点
split_index = int(len(my_dataset) * 0.95)

# 切分数据
train_dataset = my_dataset[:split_index]  # 前95%数据
eval_dataset = my_dataset[split_index:]   # 后5%数据

# 写入训练集文件
with open("dataset/sa_article_traindata_1216.json", "w", encoding="utf-8") as train_file:
    json.dump(train_dataset, train_file, ensure_ascii=False, indent=4)

# 写入评测集文件
with open("dataset/sa_article_evaldata_1216.json", "w", encoding="utf-8") as eval_file:
    json.dump(eval_dataset, eval_file, ensure_ascii=False, indent=4)

print("训练集和评测集写入成功！")

训练集和评测集写入成功！


In [63]:
connection.close()